In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

In [3]:
# data I/O
data = open('data/input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))

In [5]:
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 1115394 characters, 65 unique.


In [6]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [7]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [15]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [16]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [17]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

In [18]:
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 eWXzQa?J.
xnrrmLfcCQHcXAA:3oFrp;eyJr zjN?JQFlVqZCJjFG!DSaOXnbFIrk,S.-ytpKzhfHqz;emcnLt-CVqq-$tQzXV.
3kToyO&Wx-wiR;;&G3jcgSlJJCrsxhQEgswi?M;VmChqlPJhylj;vOmgQDguJBZGTzrkUluHZXHnpTGlAawb3Ba.AmWHNxOSDs&I 
----
iter 0, loss: 104.359681
----
 he awbhun eh:l
sp !nehh.a ytvv;ayfeest : .olirlhadi wnhstftMios eeepspos s adpthnregas Luot
e  .
r n ss   oacun 
A w a ctmidehzs s attiieytunyn:i
hiarte U o
li e:wey,eytTo hyauu pnukpVa'dtiik
d
 tmaap 
----
iter 100, loss: 104.436917
----
 osseyuoi evgdal
Chtl-nW!itstaonydnwocsaiygcteo,e s d  bl'nc l, cco,  eiacrFeptneseToesncrleyhtne,mheh nlooaTesennsIee,nen oi mid dnnt t
 essoes?r onytcnens ,et
itceoe Uuton a s heenw!h pnnseerc !ttoN  
----
iter 200, loss: 102.631129
----
 s,dCA
TIame
gamipw oiiisTytemrII
ylntesSfNVnbto usR.nr my aotioaCi'p;N, TtRCulnbylawthboCarlioA
elat tt rCt Ch a RlAnWowh d rLn esm
apeak
hio'nlp,o eoi ;StRdtissaars!woe ytyaraRtwoWttn eet C ireAmsdse 
----
iter 300, loss: 100.892846
----
 n!h f'T tieo m'bogoya lh,e t,
 yai o

----
 sheat pa pove yowa mf asmur then sersund.

CINEUS:
Anilot yo' more ve ore rast or ham, sass on, lind ant vere ri.

Bevercer mers at her mewo puke theilesore
I ghis eritf-fend, sore rta thins ar,
Of no 
----
iter 3500, loss: 58.270291
----
 s,e tho clouut nositwy: and,
Coberlits, ark a chatise tfemant yow the tr sinet,
Tqyen batdyy if I divinte- whot hine'ced royder.

CINIUS:
MMem pore weridt mi'ce.

BENIUS:
NOy:
Ar, thar ty thed anle tr 
----
iter 3600, loss: 58.031707
----
 NUS:
Fo histhe you in Bnathey to the hace tham hace to the elis to, the  ivine: sunduss,
A lronen
Houl the at my chis
fireor
And cown sthem I Ciow, ko tirer.

BRAMNAA:

ARCOCINIU: Cou tarerer 'os ow M 
----
iter 3700, loss: 57.587666
----
 e!

MENENIUS:
Meit's'tinof.

Bis what pit whis
moan hil ir Perere, soon' ly ir spashend, mor thed it ha ke com'ins'd.

MENoRIUS:
I cith nould shemke
Af com uren, loce pyapy, wer cose wome felis ? rat  
----
iter 3800, loss: 57.110485
----
 iks aucing ithe cisuy ald de't:
Yo

----
 oute;
Forkecemy, your fuissanls encim.

BLEEN:
Trout trou.

GDEENET:
What, hore, heke. lomt with axd ins oulfses sofurt stersbley hind dod, waln more.

GLUENEET:
I to voun derse say wibbfow the eret m 
----
iter 7000, loss: 54.558949
----
 ER:
Trus moth frone arochead Jose!
'n aln,

Shid mes of ther you on knan hing sing es, said thou.
As!

TERECEClENES:
Go thee, reochees mekever nat sove Hrinut
Qore shing fore the anat!

LAOFTER:
GLpan 
----
iter 7100, loss: 54.669452
----
 taes Ands tid dist te his at hint hish bescMes,
I now hen kaven fronwind pat there A unt andik,
I A detseits, Hy you tome wavl, I leverw.

QOUEDENES:
Sy and haghe.
Foy byt'n geay And than upaf, toume, 
----
iter 7200, loss: 54.573212
----
  arers!
You het I ind ne'dt Roue bot epbate stolld citettarut?
Hous knout rivet doves!
Hit: he oulle mellf, Gfould h, you wid any, thom I' t, andyle the Evimin.

Whie, wertoun ale;
I? I ino, thatard o 
----
iter 7300, loss: 54.377894
----
 fir I wastmonruling toy lore be fo

----
 ibe tilkiet lemace'll breecure thiund me, ceard uf the Woth ald and whew? stor majlinger thaes cecimese your And thins wine!
LUveind.
znead, wefpyour.

LULBUETUHAMA:
Bouchen:
Of sutlet we om the thens 
----
iter 10500, loss: 51.634493
----
  bits celencenty'lac mealifur hand tooy on Glanenly uland,
And, befey lever, fo angrer, allqune to ghy my, a bey my ayf you to my ass your of soktseld my thence wind fOke roe had somietns givers:
By w 
----
iter 10600, loss: 51.798904
----
 , he dengee,,
I when ase camvend to you toll bell now'ur soiit ubrigtt.
Wrame tham mong!

GLOUSDKAC:
Ansgies on be a, utcisy the corum hole wigh, ta it leach Lo prave Pay to yem thot upricise.

GUCEST 
----
iter 10700, loss: 51.957898
----
 ls wow minse stostsy, wimpers, she's your o, am hath mpeecim, tho a taks?
Fal Wo troul ham that a his of sale thon you hot adt thlot Cleathy.

DKINS:
Thee, nod,
Which, I amye st' the seater matr.

HAR 
----
iter 10800, loss: 51.714786
----
  cod
Tour yivarving shviin wha

----
 saads ast
Brarmuuins bows, ealdove bpocked sus me buss, not oo thith I llomt of?
There; fure peaventi, ind he heenk,
A fise anvy:
Sle sound nearse grinep-sos?

GORBYbunss in lind this ticmash bonte; a 
----
iter 13900, loss: 52.000867
----
 t RongLy'd;
Not my foth hantins: spane, in kapur near?
I wand
Corfe tell nath Trodsh and ard
Tell, arectimy pele?
Whingyun,
Theis?

JOHOust arn I andan'd fowien
Thay ppoutcins fiin teingt brunell bath 
----
iter 14000, loss: 52.076027
----
 ing lim'drord forfeith mukt and shecound
Whis my's Gollinbne that it that kiest opreal's chishespeing,
On intere profing ound Burtinged wis bredat.
Jut notsesund not mon suqusoman'dmeable landw;
II th 
----
iter 14100, loss: 51.773253
----
 tind so krochin's veachul scedend,
A'd tham he mrumunnissse to mathese The aschevetlyssgin: meate ther Doll mags him Eist fe ramTher have hists,
Buce anthiger ver, Sromey He brvesly nom sstirse to daw 
----
iter 14200, loss: 51.708499
----
 lrs a dame,s ands, but nat, un

----
 ath coll thim 'randils gigesn age grover spa rontepy heacun therkn on kent
Shasge ay key!! peerees cink,
The stainster fores,
In dave, reaw;
The seall us sosest.

COHESRF:
Soiver ongeell louss the hin 
----
iter 17300, loss: 49.110972
----
 
I of cod the trountncurgeanginouved nourt apre;
Me bearrty me on pite,
A pay your vaty dath,
And his all thou tate.
'stilkn fare noull's saile, My o deay, is warme thou there;
Alond we now preith ith 
----
iter 17400, loss: 49.083209
----
 bus, nos taprirkse 'nerstol, in hin us heinger ay the berenciongion;
Hey end of ath hane:
end;'d this ay miard sparts, then sies, And on gring oul 'fringor for forbme.

GOHS I OF YON RICHARD YI I ROCH 
----
iter 17500, loss: 49.229373
----
 d,
And arm,
Theisamers fow ase tisting thear stiNgn fakn to hemiLest intite well;
Ander
OnUKaveear coreens, e,
It as thy andut it ut be tankee:
And?
Mange proust, to baid splaeqund porth o not my bect 
----
iter 17600, loss: 49.226976
----
 e worr foll of the hoth kite
H

----
 
O land con cowloturgore;
Tromk,
Aghast wist I lienty pnouching bivet so magmens!

JOLILT:
The midilt boverturt,
My thou wis livet, ermach whith that under hanver?
The thy is los, biout hereaing wall  
----
iter 20700, loss: 51.513021
----
 soup: youn is it your uts and My, I Lorle'd
Tou heast fis mider wath nouker
Whe dallinck:
Hat sell thad reseltor'd hatrinspertoret'd O, hate:
Wishun beponttin, prims?

Ppully;
Mords, oo me theredc!
Gr 
----
iter 20800, loss: 51.536051
----
 -en!
Thom ofabys good come,
Ant for ondy stase heas him'; bpomi! Eir the fisc'ig: ain wit,
Fold the Morese! mo, mestirioold, Roulow:
Tilldas'
BAUTOPUAN.
ERCETACIA:
Norchy thise shauld.

RAMETBE:
Thy b 
----
iter 20900, loss: 51.517604
----
 l swance nemel,'s sake?

RRCATE:
At hear theareh, sovether sucand, sove's frevet: it yourney in mpamen Fy day, that I, in that wo RAved tin thy saming thenk?

BUTICSIATERLEO:
Here weth hither exbly ow 
----
iter 21000, loss: 51.401708
----
 n will you that thour, in afk 

----
 r undar gofe, lingakTy of taket:
enn;
The elonh thee Butayes chie his bench, mainetsuoing rese un Eath died
WAow sabporraas kriTs this spathi: ther, oul,
Wakt-sole tiless hese the dayt banke criasseci 
----
iter 24100, loss: 50.607391
----
 art, me cimen good il with Withy house, snil.

ROHENRELEF:
Sun andy oFart head, and where.

UFTARt:
Thine to bess,
Agon sius to they thoy ake,
And nowere.
3ed unsese his, woth con
Weve ther.
Thy my ee 
----
iter 24200, loss: 50.560683
----
 pber'dfered dedsos,
To meonem o, uner, and sut gares, near.

GORINFARD:
Thith him him her nosd:
For I fare hingch ef my bey'd sath Petord, tourd the apbave chis jove houster ontare he but arcon me thi 
----
iter 24300, loss: 50.542870
----
 tor nead ild Srath he wasthage took?
Sy hithere or, sheerngldoe dade to fartyef the Darrame to broket,
Wrows theirthign ry.
When,
This gon, aranging.

LI:
The nowird mullous,
Wroultse forest hingssger 
----
iter 24400, loss: 50.548887
----
 speith of fuly no buplect'lath

----
 ouhort duc to brysibratenton?
And stpeam sucher shepe wich Crive this Mrovear'd comtine'd with My protcengstor;
And will the wore as thereisens!

KING ED:
Fidish that of allif He combove, plond withel 
----
iter 27500, loss: 48.386859
----
  yeert, the gats roter trearch carmawar,

KING:
I matiar thy thee, fare Sid gevend telfend crarsee, cemea he fother fard wing gove broe.

WARWING YICK:
For hand anss?;
Ting hiy qus.

OXFORD
NENF CIVUT 
----
iter 27600, loss: 48.114367
----
 issaroor.

KING RICHARD:
FOle the hall and now, aroup:
And boult haort, the gyint, hentt warpe broverige, claawh
Wis time for of sland mespait,.

WICXATEN:
And, ut nos bry you bet longres?

WARWACED:
 
----
iter 27700, loss: 48.044222
----
 wicl attpeard browhhiny kore hearun voul, ave fur hure I Mit, wimy some wing't hathte Warear ont fare o'
Will bet pright Wore beaman briegube,
With oun thy will wike.

BENWICMASFO:
Seaig all murtere,  
----
iter 27800, loss: 48.099049
----
 cloueld evere?
O hare ce wher'

KeyboardInterrupt: 